# Segmenting and Clustering Neighborhoods in Toronto

### Data Frame
###### Data obtained from wikipedia. 

In [9]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [12]:
#Reads the first table of wikipedia HTML 
Toronto_NB = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [14]:
#Drop the Borough not assigned
Toronto_NB = Toronto_NB.drop(Toronto_NB[Toronto_NB['Borough'] == "Not assigned"].index)

In [16]:
Toronto_NB.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


##### There are not Neighbourhoods with not assigned values

## Second Excersice

In [20]:
Toronto_NB.shape

(103, 3)

In [25]:
df = pd.read_csv('http://cocl.us/Geospatial_data')
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [27]:
# Inner merge by postal code
merged_inner = pd.merge(left=Toronto_NB, right=df, left_on='Postal Code', right_on='Postal Code')
merged_inner.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Third Excersice

#### Libraries for map ploting

In [31]:
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

In [42]:
map_Toronto = folium.Map(location=[43.7, -79.4], zoom_start=11)

for lat, lng, borough, neighborhood in zip(merged_inner['Latitude'], merged_inner['Longitude'], merged_inner['Borough'], merged_inner['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto) 

map_Toronto

#### Libraries for clustering

In [45]:
import seaborn as sb
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

In [46]:
print(merged_inner.groupby('Borough').size())

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
dtype: int64


#### Making the clusters

In [61]:
z = merged_inner.groupby('Borough').size()
x = np.array(z[[0]])

In [62]:
kmeans = KMeans(n_clusters=5).fit(x)
centroids = kmeans.cluster_centers_
print(centroids)

ValueError: Expected 2D array, got 1D array instead:
array=[9.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.